In [2]:
!ls

__init__.py    client.ipynb   models.py      scraper.py
__pycache__    data           processtext.py


### Standard module imports:

In [3]:
from collections import defaultdict
import importlib
import re


### Project imports:

In [4]:
#import scraper
import processtext 

In [ ]:
hist = defaultdict(int)

for transcript in transcripts:
    hist[transcript['show']] += 1
    
hist

In [144]:
importlib.reload(processtext)

transcripts = processtext.main()

tr = transcripts[420]
tr['transcript']

{'Hannity': 120, 'Ingraham': 120, 'Maddow': 134, 'ODonnel': 132}


'\nTHE LAST WORD WITH LAWRENCE O`DONNELLMarch 16, 2018Guest: Barry McCaffrey, Harry Litman, Ned Price, David Cay Johnston, Ruth Marcus, Benjamin Wittes, Jill Wine-Banks; Michael Avenatti\nRACHEL MADDOW, MSNBC HOST: Now, it`s time for THE LAST WORD WITH LAWRENCE O`DONNELL. Good evening, Lawrence.\nLAWRENCE O`DONNELL, MSNBC HOST: Good evening, Rachel. So if you get a standing ovation for pleading guilty, what do you get for pleading not guilty?\nMADDOW: Yes.\nO`DONNELL: I mean, that`s – you get a car or something? You must get a prize.\nMADDOW: If Paul Manafort was able – well, despite the ankle bracelets, if Paul Manafort was able to walk into that room, I imagine they would have like – they would have declared him a saint. I don`t know.\nO`DONNELL: Rachel, we got this extraordinary news tonight about the President trying to move the Stormy Daniels lawsuit into federal court, which, of course, is the place where he appoints the judges.\nStormy Daniels` lawyer is actually on an airplane 

In [145]:
episode = processtext.Episode(tr)
episode.transcript

'chael avenatti  now its time for the last word with     good evening    good evening rachel so if you get a standing ovation for pleading guilty what do you get for pleading not guilty  yes  i mean thats   you get a car or something you must get a prize  if paul manafort was able   well despite the ankle bracelets if paul manafort was able to walk into that room i imagine they would have like   they would have declared him a saint i dont know  rachel we got this extraordinary news tonight about the president trying to move the stormy daniels lawsuit into federal court which of course is the place where he appoints the judges stormy daniels lawyer is actually on an airplane at this moment we hope to hear from him as soon as he lands but i want to bring you something else which is what barry mccaffrey   general barry mccaffrey said tonight general barry mccaffrey is going to be our first guest now i just want to get your reaction to what he said before i go to the general he tweeted rel

In [146]:
documents = []
for tr in transcripts:
    episode = processtext.Episode(tr)
    documents.append(episode.transcript)


In [37]:
len(documents)

506

## Brendan's example:

In [38]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

In [39]:
count_vectorizer.fit(documents)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [41]:
counts = count_vectorizer.transform(documents).transpose()

In [42]:
counts.shape

(900865, 506)

In [43]:
corpus = matutils.Sparse2Corpus(counts)

id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [45]:
len(id2word)

900865

In [62]:
lda = models.LdaModel(corpus=corpus, 
                      num_topics=4, 
                      minimum_probability=0.03, 
                      id2word=id2word, 
                      passes=10)

2018-05-28 19:17:26,337 : INFO : using symmetric alpha at 0.25
2018-05-28 19:17:26,339 : INFO : using symmetric eta at 0.25
2018-05-28 19:17:26,493 : INFO : using serial LDA version on this node
2018-05-28 19:17:26,990 : INFO : running online (multi-pass) LDA training, 4 topics, 10 passes over the supplied corpus of 506 documents, updating model once every 506 documents, evaluating perplexity every 506 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-28 19:17:46,969 : INFO : -14.749 per-word bound, 27541.7 perplexity estimate based on a held-out corpus of 506 documents with 3457514 words
2018-05-28 19:17:46,970 : INFO : PROGRESS: pass 0, at document #506/506
2018-05-28 19:17:50,462 : INFO : topic #0 (0.250): 0.005*"trump" + 0.003*"president" + 0.003*"know" + 0.003*"going" + 0.003*"people" + 0.002*"think" + 0.002*"right" + 0.002*"said" + 0.002*"don" + 0.002*"just"
2018-05-28 19:17:50,478 : INFO : topic #1 (0.250): 0.004*"president" + 0.004*"know" + 0.003*"trump"

2018-05-28 19:20:25,564 : INFO : topic diff=0.056284, rho=0.353553
2018-05-28 19:20:49,644 : INFO : -11.412 per-word bound, 2724.4 perplexity estimate based on a held-out corpus of 506 documents with 3457514 words
2018-05-28 19:20:49,645 : INFO : PROGRESS: pass 7, at document #506/506
2018-05-28 19:20:53,098 : INFO : topic #0 (0.250): 0.001*"trump" + 0.001*"president" + 0.001*"going" + 0.001*"know" + 0.001*"house" + 0.001*"just" + 0.001*"right" + 0.001*"new" + 0.001*"investigation" + 0.001*"people"
2018-05-28 19:20:53,118 : INFO : topic #1 (0.250): 0.002*"president" + 0.002*"know" + 0.002*"people" + 0.002*"going" + 0.002*"trump" + 0.002*"just" + 0.002*"think" + 0.001*"like" + 0.001*"right" + 0.001*"don"
2018-05-28 19:20:53,146 : INFO : topic #2 (0.250): 0.006*"trump" + 0.005*"president" + 0.004*"know" + 0.004*"going" + 0.004*"think" + 0.004*"people" + 0.003*"just" + 0.003*"right" + 0.003*"like" + 0.003*"don"
2018-05-28 19:20:53,166 : INFO : topic #3 (0.250): 0.005*"president" + 0.004*"

In [73]:
lda.print_topics()

2018-05-28 20:46:32,510 : INFO : topic #0 (0.250): 0.001*"trump" + 0.001*"president" + 0.001*"going" + 0.001*"house" + 0.001*"know" + 0.001*"russia" + 0.000*"just" + 0.000*"investigation" + 0.000*"new" + 0.000*"right"
2018-05-28 20:46:32,526 : INFO : topic #1 (0.250): 0.002*"president" + 0.002*"people" + 0.002*"know" + 0.002*"going" + 0.002*"trump" + 0.002*"just" + 0.002*"think" + 0.001*"like" + 0.001*"right" + 0.001*"don"
2018-05-28 20:46:32,543 : INFO : topic #2 (0.250): 0.006*"trump" + 0.005*"president" + 0.004*"know" + 0.004*"going" + 0.004*"think" + 0.004*"people" + 0.003*"just" + 0.003*"right" + 0.003*"like" + 0.003*"don"
2018-05-28 20:46:32,566 : INFO : topic #3 (0.250): 0.005*"president" + 0.004*"trump" + 0.003*"know" + 0.003*"just" + 0.002*"going" + 0.002*"right" + 0.002*"people" + 0.002*"think" + 0.002*"fbi" + 0.002*"investigation"


[(0,
  '0.001*"trump" + 0.001*"president" + 0.001*"going" + 0.001*"house" + 0.001*"know" + 0.001*"russia" + 0.000*"just" + 0.000*"investigation" + 0.000*"new" + 0.000*"right"'),
 (1,
  '0.002*"president" + 0.002*"people" + 0.002*"know" + 0.002*"going" + 0.002*"trump" + 0.002*"just" + 0.002*"think" + 0.001*"like" + 0.001*"right" + 0.001*"don"'),
 (2,
  '0.006*"trump" + 0.005*"president" + 0.004*"know" + 0.004*"going" + 0.004*"think" + 0.004*"people" + 0.003*"just" + 0.003*"right" + 0.003*"like" + 0.003*"don"'),
 (3,
  '0.005*"president" + 0.004*"trump" + 0.003*"know" + 0.003*"just" + 0.002*"going" + 0.002*"right" + 0.002*"people" + 0.002*"think" + 0.002*"fbi" + 0.002*"investigation"')]

## Try Gensim (w NLTK) according to SO post: https://stackoverflow.com/questions/34207097/print-document-wise-topics-in-gensim

In [152]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [192]:
new_stopwords = list(STOPWORDS)
new_stopwords = new_stopwords + ['s', 't', 'thei', 'thi', 'hi', 'wa', 'ha', 'o', 'ar', 'right', 'sai', 'becaus',
                                'new', 'think', 'know', 'like', 'michael', 'said', 'time', 'tonight', 'dont', 
                                 'people', 'peopl', 'want', 'todai', 'thing', 'look', 'let', 'come', 'wai']

## Notes w Helen:
* WOrdnet Lemmatizer (NLTK)

In [193]:
new_stopwords

['them',
 'moreover',
 'might',
 'nor',
 'not',
 'bottom',
 'several',
 'does',
 'found',
 'whatever',
 'together',
 'whereafter',
 'at',
 'becomes',
 'nobody',
 'the',
 'doesn',
 'empty',
 'four',
 'out',
 'how',
 'least',
 'fill',
 'before',
 'these',
 'unless',
 'didn',
 'front',
 'a',
 'yourselves',
 'for',
 'down',
 'what',
 'even',
 'really',
 'alone',
 'except',
 'please',
 'due',
 'meanwhile',
 'that',
 'be',
 'noone',
 'himself',
 'nothing',
 'hers',
 'eg',
 'too',
 'interest',
 'we',
 'hasnt',
 'sometimes',
 'mill',
 'than',
 'twenty',
 'while',
 'my',
 'he',
 'why',
 'and',
 'did',
 'bill',
 'an',
 'mostly',
 'already',
 'amount',
 'formerly',
 'after',
 'whereby',
 'anyone',
 'enough',
 'cannot',
 'seem',
 'seems',
 'upon',
 'who',
 'up',
 'under',
 'eleven',
 'other',
 'serious',
 'beside',
 'herself',
 'most',
 'thereupon',
 'side',
 'do',
 'de',
 'find',
 'you',
 'last',
 'must',
 'full',
 'cry',
 'am',
 'forty',
 'therein',
 'ourselves',
 'would',
 'others',
 'almost',


In [194]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list

# Create p_stemmer of class PorterStemmer
porter_stemmer = PorterStemmer()


num_topics=2;

# list for tokenized documents in loop
texts = []

# loop through document list
for doc in documents:

    
    # clean and tokenize document string
    tokens = tokenizer.tokenize(doc)

    #tokens = [token for token in tokens if token not in STOPWORDS]

    # stem tokens
    tokens = [porter_stemmer.stem(token) for token in tokens]
    #print("printing stemmed tokens")
    #print(stemmed_tokens)
    
    # remove stop words from tokens
    tokens = [token for token in tokens if token not in new_stopwords
    #print(stopped_tokens)


    # add tokens to list
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
print("printing each token/words along with unique integer id..")
#print(dictionary.token2id)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print("printing sample bag of words")
#print(corpus[0])

2018-05-29 10:57:39,235 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-29 10:57:40,657 : INFO : built Dictionary(23352 unique tokens: ['100', '15', '18', '20', '2008']...) from 506 documents (total 1543786 corpus positions)


printing each token/words along with unique integer id..
printing sample bag of words


In [196]:
# generate LDA model
ldamodel = LdaModel(corpus, num_topics=18, id2word = dictionary, passes=7)
#print(ldamodel.print_topics(num_topics=3, num_words=3))
#print ldamodel.top_topics(corpus,2)
print(ldamodel.show_topics(num_topics=2, num_words=3, log=False, formatted=True))
print(ldamodel.show_topics())
print("from for loop.")
for topic in ldamodel.show_topics(len(dictionary)):
    print(topic)

2018-05-29 10:57:41,858 : INFO : using symmetric alpha at 0.05555555555555555
2018-05-29 10:57:41,860 : INFO : using symmetric eta at 0.05555555555555555
2018-05-29 10:57:41,873 : INFO : using serial LDA version on this node
2018-05-29 10:57:41,944 : INFO : running online (multi-pass) LDA training, 18 topics, 7 passes over the supplied corpus of 506 documents, updating model once every 506 documents, evaluating perplexity every 506 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-29 10:57:41,945 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-05-29 10:57:46,204 : INFO : -11.050 per-word bound, 2119.6 perplexity estimate based on a held-out corpus of 506 documents with 1543786 words
2018-05-29 10:57:46,205 : INFO : PROGRESS: pass 0, at document #506/506
2018-05-29 10:57:47,949 : INFO : topic #7 (0.056): 0.013*"trump" + 0.011*"presid" + 0.006*"hous" + 0.005*"year" + 0.005*"d

2018-05-29 10:58:22,511 : INFO : topic #6 (0.056): 0.014*"presid" + 0.013*"trump" + 0.009*"hous" + 0.008*"book" + 0.007*"white" + 0.007*"bannon" + 0.006*"veri" + 0.005*"report" + 0.005*"realli" + 0.004*"steve"
2018-05-29 10:58:22,513 : INFO : topic #17 (0.056): 0.011*"presid" + 0.008*"hous" + 0.008*"white" + 0.008*"chemic" + 0.008*"porter" + 0.008*"john" + 0.007*"kelli" + 0.007*"weapon" + 0.006*"veri" + 0.006*"syria"
2018-05-29 10:58:22,516 : INFO : topic diff=0.744754, rho=0.377964
2018-05-29 10:58:27,875 : INFO : -7.747 per-word bound, 214.8 perplexity estimate based on a held-out corpus of 506 documents with 1543786 words
2018-05-29 10:58:27,876 : INFO : PROGRESS: pass 6, at document #506/506
2018-05-29 10:58:30,114 : INFO : topic #4 (0.056): 0.008*"presid" + 0.006*"trump" + 0.006*"francisco" + 0.005*"democrat" + 0.005*"realli" + 0.005*"san" + 0.004*"veri" + 0.004*"year" + 0.004*"gui" + 0.004*"state"
2018-05-29 10:58:30,116 : INFO : topic #13 (0.056): 0.008*"trump" + 0.004*"presid" 

[(17, '0.011*"presid" + 0.009*"hous" + 0.008*"white"'), (9, '0.018*"presid" + 0.016*"trump" + 0.005*"investig"')]
[(2, '0.005*"presid" + 0.004*"trump" + 0.002*"investig" + 0.002*"veri" + 0.002*"report" + 0.002*"realli" + 0.002*"state" + 0.002*"talk" + 0.002*"happen" + 0.002*"republican"'), (7, '0.013*"trump" + 0.011*"presid" + 0.007*"republican" + 0.006*"democrat" + 0.006*"hous" + 0.006*"senat" + 0.006*"vote" + 0.006*"veri" + 0.006*"year" + 0.005*"realli"'), (14, '0.008*"presid" + 0.008*"trump" + 0.008*"fbi" + 0.007*"investig" + 0.005*"inform" + 0.005*"veri" + 0.004*"clinton" + 0.004*"im" + 0.004*"talk" + 0.004*"democrat"'), (5, '0.016*"presid" + 0.013*"trump" + 0.009*"report" + 0.008*"hous" + 0.008*"investig" + 0.007*"white" + 0.006*"russian" + 0.006*"veri" + 0.006*"russia" + 0.005*"fbi"'), (11, '0.010*"trump" + 0.009*"presid" + 0.008*"democrat" + 0.007*"state" + 0.006*"republican" + 0.006*"talk" + 0.005*"veri" + 0.005*"im" + 0.005*"year" + 0.005*"realli"'), (1, '0.014*"presid" + 0.01

In [198]:
ldamodel.

<bound method LdaModel.get_term_topics of <gensim.models.ldamodel.LdaModel object at 0x1a41618898>>

In [166]:
tokens = ['citizen',
 'investig',
 'claim',
 's',
 'investig',
 'hallmark',
 'thi',
 'administr',
 'ha',
 'lie',
 'contact',
 'russia',
 'fact',
 'lie',]
tokens = [token for token in tokens if token not in new_stopwords]

tokens

['citizen',
 'investig',
 'claim',
 'investig',
 'hallmark',
 'administr',
 'lie',
 'contact',
 'russia',
 'fact',
 'lie']

### Try out Spacy tools to lemmatize before LDA:

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list

# Create p_stemmer of class PorterStemmer
porter_stemmer = PorterStemmer()


num_topics=2;

# list for tokenized documents in loop
texts = []

# loop through document list
for doc in documents:

    doc = nlp(tr['transcript'])

    for token in doc:
    
    # clean and tokenize document string
    tokens = tokenizer.tokenize(doc)

    #tokens = [token for token in tokens if token not in STOPWORDS]

    # stem tokens
    tokens = [porter_stemmer.stem(token) for token in tokens]
    #print("printing stemmed tokens")
    #print(stemmed_tokens)
    
    # remove stop words from tokens
    tokens = [token for token in tokens if token not in new_stopwords
    #print(stopped_tokens)


    # add tokens to list
    texts.append(tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
print("printing each token/words along with unique integer id..")
#print(dictionary.token2id)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print("printing sample bag of words")
#print(corpus[0])

In [150]:
documents[1]

'      this is a rush transcript from   may 18 2018 this copy may not be in its final form and may be updated  all right thanks tucker  welcome to     we have breaking news all over the place tonight  the deep state abuse of power that we have been uncovering for over a year is even tonight far worse than any of us could have ever imagined this is watergate on steroids  remember the nixon scandal it was about a thirdrate burglary this is a deep state political hit job now spying on an opposition party campaign fbi spies during an election     and it gets worse the deep state isnt even trying to hide it and what they did they are now bragging about it  the former obama director of national intelligence james clapper is actually saying spying on the trump campaign was a good thing has he ever heard of the constitution were going to unpack all the stunning new details tonight now it will make you question how this could ever happen in the united states of america  and also tonight a deep 

## Spacy:

In [199]:
import spacy

nlp = spacy.load('en')

doc = nlp(tr['transcript'])

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)


 
 SPACE   
 False False
Show show PROPN NNP ROOT Xxxx True False
: : PUNCT : punct : False False
THE the DET DT det XXX True False
LAST last ADJ JJ amod XXXX True False
WORD word NOUN NN appos XXXX True False
WITH with ADP IN prep XXXX True False
LAWRENCE lawrence PROPN NNP compound XXXX True False
O’DONNELLDate o’donnelldate NOUN NN pobj X’XXXXxxx False False
: : PUNCT : punct : False False
November november PROPN NNP appos Xxxxx True False
1 1 NUM CD nummod d False False
, , PUNCT , punct , False False
2017Guest 2017guest NUM CD punct ddddXxxxx False False
: : PUNCT : punct : False False
George george PROPN NNP compound Xxxxx True False
F. f. PROPN NNP compound X. False False
Will will VERB MD intj Xxxx True False

 
 SPACE   
 False False
LAWRENCE lawrence PROPN NNP compound XXXX True False
O’DONNELL o’donnell VERB VB dep X’XXXX False False
, , PUNCT , punct , False False
MSNBC msnbc PROPN NNP compound XXXX True False
HOST host PROPN NNP ROOT XXXX True False
: : PUNCT : punct : Fa

( ( PUNCT -LRB- punct ( False False
LAUGHTER laughter NOUN NN ROOT XXXX True False
) ) PUNCT -RRB- punct ) False False

 
 SPACE   
 False False
O’DONNELL o’donnell NOUN NN ROOT X’XXXX False False
: : PUNCT : punct : False False
If if ADP IN mark Xx True False
so so ADV RB advcl xx True True
, , PUNCT , punct , False False
if if ADP IN mark xx True True
so so ADV RB advcl xx True True
, , PUNCT , punct , False False
have have VERB VBP aux xxxx True True
you -PRON- PRON PRP nsubj xxx True True
cooperated cooperate VERB VBN ROOT xxxx True False
with with ADP IN prep xxxx True True
these these DET DT det xxxx True True
inquiries inquiry NOUN NNS pobj xxxx True False
? ? PUNCT . punct ? False False

 
 SPACE   
 False False
How how ADV WRB advmod Xxx True False
’s ’ VERB VBZ ROOT ’x False False
that that DET DT nsubj xxxx True True
for for ADP IN prep xxx True True
question question NOUN NN pobj xxxx True False
one one NUM CD nsubj xxx True True
of of ADP IN prep xx True True
a a DET DT de

of of ADP IN prep xx True True
the the DET DT det xxx True True
doubt doubt NOUN NN pobj xxxx True False
about about ADP IN prep xxxx True True
what what NOUN WP dobj xxxx True True
he -PRON- PRON PRP nsubj xx True True
meant mean VERB VBD pcomp xxxx True False
when when ADV WRB advmod xxxx True True
he -PRON- PRON PRP nsubj xx True True
called call VERB VBD advcl xxxx True False
Congresswoman congresswoman PROPN NNP compound Xxxxx True False
Wilson wilson PROPN NNP dobj Xxxxx True False
an an DET DT det xx True True
empty empty ADJ JJ amod xxxx True True
barrel barrel NOUN NN oprd xxxx True False
and and CCONJ CC cc xxx True True
what what NOUN WP dobj xxxx True True
he -PRON- PRON PRP nsubj xx True True
meant mean VERB VBD conj xxxx True False
when when ADV WRB advmod xxxx True True
he -PRON- PRON PRP nsubj xx True True
called call VERB VBD advcl xxxx True False
Robert robert PROPN NNP compound Xxxxx True False
E. e. PROPN NNP compound X. False False
Lee lee PROPN NNP dobj Xxx True F

to to ADP IN prep xx True True
the the DET DT det xxx True True
Mueller mueller PROPN NNP compound Xxxxx True False
appointment appointment NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
according accord VERB VBG prep xxxx True False
to to ADP IN prep xx True True
a a DET DT det x True True
source source NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
who who NOUN WP attr xxx True True
could could VERB MD aux xxxx True True
that that DET DT nsubj xxxx True True
be be VERB VB relcl xx True True
? ? PUNCT . punct ? False False
A a DET DT det X True False
source source NOUN NN nsubjpass xxxx True False
briefed brief VERB VBN acl xxxx True False
on on ADP IN prep xx True True
Steve steve PROPN NNP compound Xxxxx True False
Bannon bannon PROPN NNP pobj Xxxxx True False
’s ’s PART POS auxpass ’x False False
call call NOUN NN ROOT xxxx True True
with with ADP IN prep xxxx True True
Donald donald PROPN NNP compound Xxxxx True False
Trump trump PROPN NNP pobj Xxxxx Tr

Stone stone NOUN NN nsubj Xxxxx True False
wants want VERB VBZ ROOT xxxx True False
Trump trump PROPN NNP nsubj Xxxxx True False
to to PART TO aux xx True True
call call VERB VB ccomp xxxx True True
for for ADP IN prep xxx True True
a a DET DT det x True True
special special ADJ JJ amod xxxx True False
prosecutor prosecutor NOUN NN pobj xxxx True False
to to PART TO aux xx True True
investigate investigate VERB VB acl xxxx True False
Hillary hillary PROPN NNP compound Xxxxx True False
Clinton clinton PROPN NNP poss Xxxxx True False
’s ’s PART POS case ’x False False
role role NOUN NN dobj xxxx True False
in in ADP IN prep xx True True
approving approve VERB VBG pcomp xxxx True False
the the DET DT det xxx True True
controversial controversial ADJ JJ amod xxxx True False
Uranium uranium PROPN NNP nmod Xxxxx True False
One one NUM CD nummod Xxx True False
deal deal NOUN NN dobj xxxx True False
that that ADJ WDT nsubjpass xxxx True True
’s ’ VERB VBZ auxpass ’x False False
been be VERB VB

...

, , PUNCT , punct , False False
try try VERB VB ROOT xxx True False
to to PART TO aux xx True True
imagine imagine VERB VB xcomp xxxx True False
what what NOUN WP nsubj xxxx True True
’s ’ VERB VBZ aux ’x False False
going go VERB VBG ccomp xxxx True False
on on PART RP prt xx True True
in in ADP IN prep xx True True
his -PRON- ADJ PRP$ poss xxx True True
mind mind NOUN NN pobj xxxx True False
. . PUNCT . punct . False False
When when ADV WRB advmod Xxxx True False
’s ’s NOUN NN aux ’x False False
driving drive VERB VBG ROOT xxxx True False
that that DET DT dobj xxxx True True
? ? PUNCT . punct ? False False

 
 SPACE   
 False False
Why why ADV WRB advmod Xxx True False
the the DET DT det xxx True True
mentioned mention VERB VBN ROOT xxxx True False
of of ADP IN prep xx True True
Obama obama PROPN NNP pobj Xxxxx True False
in in ADP IN prep xx True True
this this DET DT det xxxx True True
starkly starkly ADJ JJ advmod xxxx True False
negative negative ADJ JJ amod xxxx True False
story

to to PART TO aux xx True True
apologize apologize VERB VB relcl xxxx True False
as as ADV RB advmod xx True True
soon soon ADV RB advmod xxxx True False
as as ADP IN prep xx True True
everything everything NOUN NN pobj xxxx True True
he -PRON- PRON PRP nsubj xx True True
said say VERB VBD relcl xxxx True False
was be VERB VBD auxpass xxx True True
proven prove VERB VBN ccomp xxxx True False
untrue untrue ADJ JJ oprd xxxx True False
the the DET DT det xxx True True
very very ADV RB advmod xxxx True True
next next ADJ JJ amod xxxx True True
day day NOUN NN npadvmod xxx True False
. . PUNCT . punct . False False

 
 SPACE   
 False False
And and CCONJ CC cc Xxx True False
he -PRON- PRON PRP nsubj xx True True
has have VERB VBZ aux xxx True True
spent spend VERB VBN ROOT xxxx True False
every every DET DT det xxxx True True
day day NOUN NN npadvmod xxx True False
since since ADP IN prep xxxx True True
then then ADV RB pcomp xxxx True True
teaching teach VERB VBG pcomp xxxx True False
us -

back back ADV RB advmod xxxx True True
in in ADP IN prep xx True True
those those DET DT det xxxx True True
days day NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False

 
 SPACE   
 False False
( ( PUNCT -LRB- punct ( False False
END end NOUN NN compound XXX True False
VIDEO video PROPN NNP compound XXXX True False
CLIP clip NOUN NN ROOT XXXX True False
) ) PUNCT -RRB- punct ) False False

 
 SPACE   
 False False
O’DONNELL o’donnell NOUN NN ROOT X’XXXX False False
: : PUNCT : punct : False False
John john PROPN NNP compound Xxxx True False
Kelly kelly PROPN NNP nsubj Xxxxx True False
the the DET DT det xxx True True
amateur amateur ADJ JJ amod xxxx True False
historian historian NOUN NN appos xxxx True False
is be VERB VBZ ROOT xx True True
full full ADJ JJ acomp xxxx True True
of of ADP IN prep xx True True
poisonous poisonous ADJ JJ amod xxxx True False
falsehoods falsehood NOUN NNS pobj xxxx True False
about about ADP IN prep xxxx True True
American american ADJ JJ amod 

racist racist ADJ JJ amod xxxx True False
time time NOUN NN pobj xxxx True False
and and CCONJ CC cc xxx True True
place place NOUN NN conj xxxx True False
and and CCONJ CC cc xxx True True
who who NOUN WP nsubj xxx True True
never never ADV RB neg xxxx True True
found find VERB VBD conj xxxx True False
the the DET DT det xxx True True
public public ADJ JJ amod xxxx True False
way way NOUN NN dobj xxx True False
that that ADJ WDT pobj xxxx True True
we -PRON- PRON PRP nsubj xx True True
know know VERB VBP relcl xxxx True False
of of ADP IN prep xx True True
to to PART TO aux xx True True
take take VERB VB pcomp xxxx True True
a a DET DT det x True True
stand stand NOUN NN dobj xxxx True False
against against ADP IN prep xxxx True True
the the DET DT det xxx True True
racism racism NOUN NN pobj xxxx True False
of of ADP IN prep xx True True
his -PRON- ADJ PRP$ poss xxx True True
neighborhood neighborhood NOUN NN pobj xxxx True False
and and CCONJ CC cc xxx True True
of of ADP IN conj xx

attack attack NOUN NN pobj xxxx True False
in in ADP IN prep xx True True
lower low ADJ JJR amod xxxx True False
Manhattan manhattan PROPN NNP pobj Xxxxx True False
. . PUNCT . punct . False False
He -PRON- PRON PRP nsubj Xx True False
shot shoot VERB VBD ROOT xxxx True False
the the DET DT det xxx True True
suspect suspect NOUN NN dobj xxxx True False
in in ADP IN prep xx True True
the the DET DT det xxx True True
abdomen abdomen NOUN NN pobj xxxx True False
and and CCONJ CC cc xxx True True
brought bring VERB VBD conj xxxx True False
him -PRON- PRON PRP dobj xxx True True
down down PART RP prt xxxx True True
. . PUNCT . punct . False False
Today today NOUN NN compound Xxxxx True False
officer officer NOUN NN nsubj xxxx True False
Nash nash PROPN NNP appos Xxxx True False
said say VERB VBD ROOT xxxx True False
this this DET DT dobj xxxx True True
. . PUNCT . punct . False False

 
 SPACE   
 False False
( ( PUNCT -LRB- punct ( False False
BEGIN begin VERB VB ROOT XXXX True False
VIDEO

In [215]:
for i in range(85,200):
    token  = doc[i]
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
              token.shape_, token.is_alpha, token.is_stop)

reasons reason NOUN NNS pobj xxxx True False
is be VERB VBZ ROOT xx True True
that that ADP IN mark xxxx True True
now now ADV RB advmod xxx True True
that that ADP IN mark xxxx True True
we -PRON- PRON PRP nsubj xx True True
know know VERB VBP intj xxxx True False
he -PRON- PRON PRP nsubj xx True True
’s ’ VERB VBZ ccomp ’x False False
part part NOUN NN attr xxxx True True
of of ADP IN prep xx True True
the the DET DT det xxx True True
special special ADJ JJ amod xxxx True False
prosecutor prosecutor NOUN NN compound xxxx True False
’s ’s PART POS compound ’x False False
investigation investigation NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
now now ADV RB advmod xxx True True
that that ADP IN mark xxxx True True
his -PRON- ADJ PRP$ poss xxx True True
e e NOUN NN compound x True False
- - PUNCT HYPH punct - False False
mails mail NOUN NNS nsubjpass xxxx True False
have have VERB VBP aux xxxx True True
been be VERB VBN auxpass xxxx True True
revealed reveal VERB VBN cc

In [206]:
doc[50]

that

## Gensim Tutorial 1: https://radimrehurek.com/gensim/tut1.html

In [20]:
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "/var/folders/f4/6zkljh491nq9hfnrnlplkryw0000gn/T" will be used to save temporary dictionary and corpus.


In [115]:
from nltk.tokenize.regexp import RegexpTokenizer

from gensim import corpora, models, similarities, matutils
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.porter import PorterStemmer
from gensim.models.ldamodel import LdaModel


from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

from pprint import pprint  # pretty-printer

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [22]:
class MyCorpus(object):
     def __iter__(self):
        for doc in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.split())

In [26]:
documents_ = []
for tr in transcripts:
    episode = processtext.Episode(tr)
    documents_.append(episode.transcript)

In [99]:
doc = documents_[0]
doc[:150]

tokens = tokenizer.tokenize(doc)
tokens

['close',
 'video',
 'strassel',
 'chaffetz',
 'on',
 'claims',
 'of',
 'trump',
 'campaign',
 'surveillance',
 'the',
 'wall',
 'street',
 'journal',
 's',
 'kimberly',
 'strassel',
 'and',
 'fox',
 'news',
 'contributor',
 'jason',
 'chaffetz',
 'weigh',
 'in',
 'on',
 'this',
 'is',
 'a',
 'rush',
 'transcript',
 'from',
 'may',
 '21',
 '2018',
 'this',
 'copy',
 'may',
 'not',
 'be',
 'in',
 'its',
 'final',
 'form',
 'and',
 'may',
 'be',
 'updated',
 'yes',
 'can',
 'we',
 'pan',
 'and',
 'a',
 'little',
 'they',
 'have',
 'us',
 'back',
 'so',
 'far',
 'i',
 'can',
 't',
 'even',
 'see',
 'where',
 'are',
 'you',
 'if',
 'we',
 'could',
 'just',
 'pan',
 'in',
 'james',
 'and',
 'help',
 'us',
 'out',
 'here',
 'it',
 's',
 'too',
 'far',
 'away',
 'come',
 'on',
 'come',
 'on',
 'in',
 'that',
 's',
 'all',
 'right',
 'don',
 't',
 'worry',
 'people',
 'at',
 'home',
 'understand',
 'is',
 'a',
 'live',
 'television',
 'thank',
 'you',
 'tucker',
 'good',
 'luck',
 'man',
 'gre

In [95]:
tokenizer = RegexpTokenizer(r'\w+')
porter_stemmer = PorterStemmer()

In [ ]:
for doc in documents_:

    # clean and tokenize document string
    
    tokens = tokenizer.tokenize(doc)

    # remove stop words from tokens
    stopped_tokens = [doc for doc in tokens if not doc in STOPWORDS]
    #print(stopped_tokens)

    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in STOPWORDS]
    #print("printing stemmed tokens")
    #print(stemmed_tokens)

    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
print("printing each token/words along with unique integer id..")
print(dictionary.token2id)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print("printing sample bag of words")
print(corpus[0])

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)
#print(ldamodel.print_topics(num_topics=3, num_words=3))
#print ldamodel.top_topics(corpus,2)
print(ldamodel.show_topics(num_topics=2, num_words=3, log=False, formatted=True))
print(ldamodel.show_topics())
print("from for loop.")
for i in ldamodel.show_topics(len(dictionary)):
    print i

In [88]:
#stoplist = set('for a of the and to in'.split())

texts = [[word for word in document.split() if word not in STOPWORDS]
         for document in documents_]

# remove words that appear only once
from collections import defaultdict

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]



In [89]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/gensim_transcripts.dict')  # store the dictionary, for future reference

2018-05-28 21:04:01,520 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-28 21:04:02,943 : INFO : built Dictionary(25957 unique tokens: ["'", "'we", '100', '15', '18']...) from 506 documents (total 1753080 corpus positions)
2018-05-28 21:04:02,944 : INFO : saving Dictionary object under /tmp/gensim_transcripts.dict, separately None
2018-05-28 21:04:02,958 : INFO : saved /tmp/gensim_transcripts.dict


In [65]:
dictionary.token2id

{"'": 0,
 "'we": 1,
 '100': 2,
 '15': 3,
 '18': 4,
 '20': 5,
 '2008': 6,
 '2016': 7,
 '2017': 8,
 '2018': 9,
 '21': 10,
 '21st': 11,
 '35': 12,
 '400000': 13,
 '5': 14,
 '50': 15,
 '500': 16,
 '60': 17,
 '7th': 18,
 '99': 19,
 "abedin's": 20,
 'able': 21,
 'about': 22,
 'absolute': 23,
 'abuse': 24,
 'abuses': 25,
 'acceptable': 26,
 'accepted': 27,
 'access': 28,
 'accountable': 29,
 'accusations': 30,
 'accusing': 31,
 'acid': 32,
 'acted': 33,
 'acting': 34,
 'action': 35,
 'actions': 36,
 'activity': 37,
 'actors': 38,
 'actually': 39,
 'adam': 40,
 'addresses': 41,
 'administration': 42,
 'admit': 43,
 'adviser': 44,
 'advisors': 45,
 'afraid': 46,
 'after': 47,
 'again': 48,
 'against': 49,
 'agencies': 50,
 'agenda': 51,
 'agent': 52,
 'ago': 53,
 'agree': 54,
 'agreed': 55,
 'ahead': 56,
 'aide': 57,
 'aim': 58,
 'all': 59,
 "all'": 60,
 'allegations': 61,
 'allegedly': 62,
 'allout': 63,
 'almost': 64,
 'along': 65,
 'already': 66,
 'also': 67,
 'alter': 68,
 'although': 69,
 

In [72]:
dictionary.id2token

{}

In [70]:
new_doc = documents_[0] #"Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())

In [71]:
new_vec

[(0, 2),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 5),
 (5, 1),
 (6, 1),
 (7, 5),
 (8, 3),
 (9, 3),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 2),
 (16, 2),
 (17, 1),
 (18, 2),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 41),
 (23, 2),
 (24, 6),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 2),
 (38, 4),
 (39, 3),
 (40, 3),
 (41, 1),
 (42, 12),
 (43, 3),
 (44, 1),
 (45, 1),
 (46, 2),
 (47, 10),
 (48, 7),
 (49, 2),
 (50, 1),
 (51, 2),
 (52, 1),
 (53, 2),
 (54, 1),
 (55, 1),
 (56, 4),
 (57, 2),
 (58, 1),
 (59, 69),
 (60, 1),
 (61, 1),
 (62, 2),
 (63, 2),
 (64, 1),
 (65, 1),
 (66, 4),
 (67, 18),
 (68, 1),
 (69, 1),
 (70, 4),
 (71, 2),
 (72, 2),
 (73, 4),
 (74, 7),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 18),
 (79, 3),
 (80, 1),
 (81, 1),
 (82, 7),
 (83, 2),
 (84, 12),
 (85, 2),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 4),
 (90, 1),
 (91, 1),
 (92, 2),
 (93, 1),
 (94, 57),
 (95, 1),
 (96, 1),
 (97, 3),
 (98, 1),
 (99, 31),
 

In [69]:
documents_[0]

"     close      video      strassel chaffetz on claims of trump campaign surveillance the wall street journal's kimberly strassel and fox news contributor jason chaffetz weigh in on ' '     this is a rush transcript from   may 21 2018 this copy may not be in its final form and may be updated  yes can we pan and a little they have us back so far i can't even see  where are you  if we could just pan in james and help us out here it's too far away come on come on in that's all right don't worry people at home understand is a live television thank you tucker  good luck man  great show as always     all right welcome to    tonight this is unbelievable the deep state should be totally terrified as all of their nefarious deeds that we have been telling you have been exposed  this is the worst abuse of power corruption scandal in american history it is all coming out everything we have been telling you the president rightly demanding to know just how pervasive this playing was on his campaign

In [110]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
num_topics=2;

# list for tokenized documents in loop
texts = []

# loop through document list
for doc in doc_set:

    # clean and tokenize document string
    raw = doc.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [token for token in tokens if token not in STOPWORDS]
    #print(stopped_tokens)

    # stem tokens
    stemmed_tokens = [p_stemmer.stem(stopped_token) for  stopped_token in stopped_tokens]
    #print("printing stemmed tokens")
    #print(stemmed_tokens)

    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
print("printing each token/words along with unique integer id..")
print(dictionary.token2id)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
print("printing sample bag of words")
print(corpus[0])

2018-05-28 21:23:13,110 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-28 21:23:13,112 : INFO : built Dictionary(25 unique tokens: ['brocolli', 'brother', 'eat', 'good', 'like']...) from 5 documents (total 39 corpus positions)


printing each token/words along with unique integer id..
{'brocolli': 0, 'brother': 1, 'eat': 2, 'good': 3, 'like': 4, 'mother': 5, 'basebal': 6, 'drive': 7, 'lot': 8, 'practic': 9, 'spend': 10, 'time': 11, 'blood': 12, 'caus': 13, 'expert': 14, 'health': 15, 'increas': 16, 'pressur': 17, 'suggest': 18, 'tension': 19, 'better': 20, 'feel': 21, 'perform': 22, 'school': 23, 'profession': 24}
printing sample bag of words
[(0, 2), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1)]


In [117]:
# generate LDA model
ldamodel = LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)
#print(ldamodel.print_topics(num_topics=3, num_words=3))
#print ldamodel.top_topics(corpus,2)
print(ldamodel.show_topics(num_topics=2, num_words=3, log=False, formatted=True))
print(ldamodel.show_topics())
print("from for loop.")
for topic in ldamodel.show_topics(len(dictionary)):
    print(topic)

2018-05-28 21:26:06,155 : INFO : using symmetric alpha at 0.5
2018-05-28 21:26:06,157 : INFO : using symmetric eta at 0.5
2018-05-28 21:26:06,158 : INFO : using serial LDA version on this node
2018-05-28 21:26:06,159 : INFO : running online (multi-pass) LDA training, 2 topics, 20 passes over the supplied corpus of 5 documents, updating model once every 5 documents, evaluating perplexity every 5 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-28 21:26:06,172 : INFO : -4.071 per-word bound, 16.8 perplexity estimate based on a held-out corpus of 5 documents with 39 words
2018-05-28 21:26:06,173 : INFO : PROGRESS: pass 0, at document #5/5
2018-05-28 21:26:06,184 : INFO : topic #0 (0.500): 0.080*"good" + 0.080*"brocolli" + 0.077*"health" + 0.061*"eat" + 0.058*"mother" + 0.058*"drive" + 0.055*"brother" + 0.051*"pressur" + 0.037*"blood" + 0.037*"caus"
2018-05-28 21:26:06,185 : INFO : topic #1 (0.500): 0.075*"brother" + 0.070*"drive" + 0.069*"mother" + 0.058*"lot" + 0

2018-05-28 21:26:06,371 : INFO : PROGRESS: pass 11, at document #5/5
2018-05-28 21:26:06,374 : INFO : topic #0 (0.500): 0.081*"good" + 0.081*"brocolli" + 0.081*"health" + 0.058*"eat" + 0.058*"pressur" + 0.057*"drive" + 0.057*"mother" + 0.057*"brother" + 0.035*"blood" + 0.035*"caus"
2018-05-28 21:26:06,376 : INFO : topic #1 (0.500): 0.075*"brother" + 0.075*"mother" + 0.075*"drive" + 0.072*"lot" + 0.072*"practic" + 0.072*"basebal" + 0.072*"time" + 0.072*"spend" + 0.025*"perform" + 0.025*"school"
2018-05-28 21:26:06,377 : INFO : topic diff=0.003032, rho=0.277350
2018-05-28 21:26:06,382 : INFO : -3.696 per-word bound, 13.0 perplexity estimate based on a held-out corpus of 5 documents with 39 words
2018-05-28 21:26:06,383 : INFO : PROGRESS: pass 12, at document #5/5
2018-05-28 21:26:06,387 : INFO : topic #0 (0.500): 0.081*"good" + 0.081*"brocolli" + 0.081*"health" + 0.058*"eat" + 0.058*"pressur" + 0.057*"drive" + 0.057*"mother" + 0.057*"brother" + 0.035*"blood" + 0.035*"caus"
2018-05-28 21:

[(0, '0.081*"good" + 0.081*"brocolli" + 0.081*"health"'), (1, '0.074*"brother" + 0.074*"mother" + 0.074*"drive"')]
[(0, '0.081*"good" + 0.081*"brocolli" + 0.081*"health" + 0.058*"eat" + 0.058*"pressur" + 0.057*"drive" + 0.057*"mother" + 0.057*"brother" + 0.035*"blood" + 0.035*"caus"'), (1, '0.074*"brother" + 0.074*"mother" + 0.074*"drive" + 0.072*"lot" + 0.072*"practic" + 0.072*"basebal" + 0.072*"time" + 0.072*"spend" + 0.025*"perform" + 0.025*"school"')]
from for loop.
(0, '0.081*"good" + 0.081*"brocolli" + 0.081*"health" + 0.058*"eat" + 0.058*"pressur" + 0.057*"drive" + 0.057*"mother" + 0.057*"brother" + 0.035*"blood" + 0.035*"caus"')
(1, '0.074*"brother" + 0.074*"mother" + 0.074*"drive" + 0.072*"lot" + 0.072*"practic" + 0.072*"basebal" + 0.072*"time" + 0.072*"spend" + 0.025*"perform" + 0.025*"school"')
